In [ ]:
#Count the number of days with temperature minimum above 293.15K 
import xarray as xr
import numpy as np
import os
import glob

input_pattern = 'tasAdjust_alpes_day_*.nc'
input_files = sorted(glob.glob(input_pattern))
freezing_threshold = 273.15

for input_file in input_files:
    base_name = os.path.basename(input_file)
    output_file = f"freezing_days_per_year_{base_name.split('tasAdjust_alpes_day_')[-1].replace('.nc', '')}.nc"
    print(f"\nProcessing {input_file} ...")

    # Open the dataset
    ds = xr.open_dataset(input_file)

    # Create a boolean mask for temperatures below freezing
    below_freezing = ds['tasAdjust'] < freezing_threshold

    # Group by year and count
    freezing_days_per_year = below_freezing.astype(int).groupby('time.year').sum(dim='time')

    # Make a mask for years (and grid cells) with all missing data
    count_days_per_year = ds['tasAdjust'].groupby('time.year').count(dim='time')
    missing_mask = count_days_per_year == 0  # True where there was no data at all

    # Convert to floats so NaNs can be present
    freezing_days_values = freezing_days_per_year.values.astype(float)
    # Set to NaN where there was no data
    freezing_days_values[missing_mask.values] = np.nan

    # Create a new dataset with the results
    ds_output = xr.Dataset({
        'freezing_days': (['year', 'y', 'x'], freezing_days_values)
    })
    
    # Set coordinates
    ds_output = ds_output.assign_coords({
        'year': freezing_days_per_year.year.values.astype(int),
        'x': ds.coords['x'],
        'y': ds.coords['y']
    })

    # Add attributes
    ds_output['freezing_days'].attrs['long_name'] = 'Count of days with temperature below 273.15K'
    ds_output['freezing_days'].attrs['units'] = 'days'
    ds_output['freezing_days'].attrs['threshold'] = '273.15K'
    
    ds_output['year'].attrs['long_name'] = 'year'

    # Save to NetCDF
    ds_output.to_netcdf(output_file)
    
    ds.close()